This notebook takes the hourly price data from 1h_prices and cleans the data removing outliers based on low volumes

In [0]:
from delta.tables import DeltaTable

In [0]:
# Read data from 'runescape.01_bronze.1h_prices'
df_1h_prices = spark.read.table("runescape.01_bronze.1h_prices")

In [0]:
# TODO decide if data should be filtered to most recent time before merging to improve performance

In [0]:
# filter out data that has low volumes
# low volume data is not accurate when predicting what the "real" price is
volume_limit = 10
df_1h_prices_cleansed = df_1h_prices.filter(f"avg1HourHighVolume > {volume_limit} AND avg1HourLowVolume > {volume_limit}")

In [0]:
%skip
df_1h_prices_cleansed.display()

In [0]:
# save data to bronze layer Unity Catalog table 'runescape.02_silver.1h_prices_cleansed'
targetDF = DeltaTable.forName(spark, "runescape.02_silver.1h_prices_cleansed")
dfUpdates = df_1h_prices_cleansed
# Insert only data that is not already in target table
(targetDF.alias("t")
  .merge(
    source = dfUpdates.alias("s"),
    condition = "t.id = s.id AND t.time = s.time")
  .whenNotMatchedInsertAll()
  .execute())